# Video to sentences without AI 

In [ ]:
import os
import moviepy.editor as mp
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence

def extract_audio(video_path, audio_path):
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

def transcribe_video(video_path):
    recognizer = sr.Recognizer()

    # Extract audio from video
    audio_path = "temp_audio.wav"
    extract_audio(video_path, audio_path)

    # Load audio and split on silence
    audio = AudioSegment.from_wav(audio_path)
    chunks = split_on_silence(
                                audio, 
                                min_silence_len=500,
                                silence_thresh=audio.dBFS-14,
                                keep_silence=500
                            )

    transcript = []

    try:
        for i, chunk in enumerate(chunks):
            chunk_name = f"chunk_{i}.wav"
            chunk.export(chunk_name, format="wav")

            with sr.AudioFile(chunk_name) as source:
                audio_data = recognizer.record(source)

            # Transcribe audio to text with timestamps
            result = recognizer.recognize_google(audio_data, show_all=True)

            if 'alternative' in result:
                for alternative in result['alternative']:
                    if 'transcript' in alternative:
                        transcript.append({
                            "start_time": None,
                            "end_time": None,
                            "text": alternative['transcript']
                        })

            os.remove(chunk_name)

    except sr.UnknownValueError:
        transcript.append({
            "start_time": None,
            "end_time": None,
            "text": "Google Speech Recognition could not understand audio"
        })
    except sr.RequestError as e:
        transcript.append({
            "start_time": None,
            "end_time": None,
            "text": f"Could not request results from Google Speech Recognition service; {e}"
        })
    finally:
        # Clean up the temporary audio file
        if os.path.exists(audio_path):
            os.remove(audio_path)

    return transcript

# Example usage
video_path = "V1.mp4"
transcription_result = transcribe_video(video_path)
for entry in transcription_result:
    print(f"Text: {entry['text']}")
